In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

import os
import nltk
import nltk.corpus
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# nltk.download() 
print(os.listdir(nltk.data.find("corpora")))

['abc', 'abc.zip', 'alpino', 'alpino.zip', 'biocreative_ppi', 'biocreative_ppi.zip', 'brown', 'brown.zip', 'brown_tei', 'brown_tei.zip', 'cess_cat', 'cess_cat.zip', 'cess_esp', 'cess_esp.zip', 'chat80', 'chat80.zip', 'city_database', 'city_database.zip', 'cmudict', 'cmudict.zip', 'comparative_sentences', 'comparative_sentences.zip', 'comtrans.zip', 'conll2000', 'conll2000.zip', 'conll2002', 'conll2002.zip', 'conll2007.zip', 'crubadan', 'crubadan.zip', 'dependency_treebank', 'dependency_treebank.zip', 'dolch', 'dolch.zip', 'europarl_raw', 'europarl_raw.zip', 'floresta', 'floresta.zip', 'framenet_v15', 'framenet_v15.zip', 'framenet_v17', 'framenet_v17.zip', 'gazetteers', 'gazetteers.zip', 'genesis', 'genesis.zip', 'gutenberg', 'gutenberg.zip', 'ieer', 'ieer.zip', 'inaugural', 'inaugural.zip', 'indian', 'indian.zip', 'jeita.zip', 'kimmo', 'kimmo.zip', 'knbc.zip', 'lin_thesaurus', 'lin_thesaurus.zip', 'machado.zip', 'mac_morpho', 'mac_morpho.zip', 'masc_tagged.zip', 'movie_reviews', 'movie

In [4]:
#Import movie reviews and check categories
from nltk.corpus import movie_reviews
movie_reviews.categories()

['neg', 'pos']

In [5]:
pos_rev = movie_reviews.fileids('pos')
len(pos_rev)

1000

In [6]:
neg_rev = movie_reviews.fileids('neg')
len(neg_rev)

1000

In [7]:
#List positive review files
pos_rev

['pos/cv000_29590.txt',
 'pos/cv001_18431.txt',
 'pos/cv002_15918.txt',
 'pos/cv003_11664.txt',
 'pos/cv004_11636.txt',
 'pos/cv005_29443.txt',
 'pos/cv006_15448.txt',
 'pos/cv007_4968.txt',
 'pos/cv008_29435.txt',
 'pos/cv009_29592.txt',
 'pos/cv010_29198.txt',
 'pos/cv011_12166.txt',
 'pos/cv012_29576.txt',
 'pos/cv013_10159.txt',
 'pos/cv014_13924.txt',
 'pos/cv015_29439.txt',
 'pos/cv016_4659.txt',
 'pos/cv017_22464.txt',
 'pos/cv018_20137.txt',
 'pos/cv019_14482.txt',
 'pos/cv020_8825.txt',
 'pos/cv021_15838.txt',
 'pos/cv022_12864.txt',
 'pos/cv023_12672.txt',
 'pos/cv024_6778.txt',
 'pos/cv025_3108.txt',
 'pos/cv026_29325.txt',
 'pos/cv027_25219.txt',
 'pos/cv028_26746.txt',
 'pos/cv029_18643.txt',
 'pos/cv030_21593.txt',
 'pos/cv031_18452.txt',
 'pos/cv032_22550.txt',
 'pos/cv033_24444.txt',
 'pos/cv034_29647.txt',
 'pos/cv035_3954.txt',
 'pos/cv036_16831.txt',
 'pos/cv037_18510.txt',
 'pos/cv038_9749.txt',
 'pos/cv039_6170.txt',
 'pos/cv040_8276.txt',
 'pos/cv041_21113.txt',
 

In [8]:
#Select one positive review
rev = movie_reviews.words('pos/cv000_29590.txt')
rev

['films', 'adapted', 'from', 'comic', 'books', 'have', ...]

In [9]:
#Parse and append tokens to a new list
rev_list = []
for rev in neg_rev:
    rev_text_neg = movie_reviews.words(rev)
    rev_one_string = " ".join(rev_text_neg)
    rev_one_string = rev_one_string.replace(' ,',',')
    rev_one_string = rev_one_string.replace(' .',',')
    rev_one_string = rev_one_string.replace("\' ","'")
    rev_one_string = rev_one_string.replace(" \'","'")
    rev_list.append(rev_one_string)
# rev_list

In [60]:
rev_list[1]

'the happy bastard\'s quick movie review damn that y2k bug, it\'s got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on, little do they know the power within,,, going for the gore and bringing on a few action sequences here and there, virus still feels very empty, like a movie going for all flash and no substance, we don\'t know why the crew was really out in the middle of nowhere, we don\'t know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ), and, of course, we don\'t know why donald sutherland is stumbling around drunkenly throughout, here, it\'s just " hey, let\'s chase these people around with some robots ", the acting is below average, even from the likes of curtis, you\'re more likely to get a kick out of her work in halloween h20, sutherla

In [11]:
for rev in pos_rev:
    rev_text_pos = movie_reviews.words(rev)
    rev_one_string = " ".join(rev_text_pos)
    rev_one_string = rev_one_string.replace(' ,',',')
    rev_one_string = rev_one_string.replace(' .',',')
    rev_one_string = rev_one_string.replace("\' ","'")
    rev_one_string = rev_one_string.replace(" \'","'")
    rev_list.append(rev_one_string)
# rev_list

In [12]:
len(rev_list)

2000

In [13]:
neg_targets = np.zeros((1000,),dtype = np.int)
pos_targets = np.ones((1000,),dtype = np.int)

In [14]:
target_list = []
for neg_tar in neg_targets:
    target_list.append(neg_tar)
for pos_tar in pos_targets:
    target_list.append(pos_tar)

In [15]:
len(target_list)

2000

In [16]:
y = pd.Series(target_list)

In [17]:
type(y)

pandas.core.series.Series

In [18]:
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [19]:
#Use bag of words or count vectorizer to make features
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(lowercase = True, stop_words = 'english', min_df = 2)

In [20]:
X_count_vec = count_vec.fit_transform(rev_list)

In [21]:
X_count_vec.shape

(2000, 23784)

In [22]:
#All X features names in count_vectorizer
X_names = count_vec.get_feature_names()
X_names

['00',
 '000',
 '007',
 '05',
 '10',
 '100',
 '1000',
 '100m',
 '101',
 '102',
 '103',
 '105',
 '106',
 '107',
 '108',
 '10th',
 '11',
 '110',
 '113',
 '115',
 '11th',
 '12',
 '126',
 '129',
 '13',
 '130',
 '132',
 '137',
 '13th',
 '14',
 '14th',
 '15',
 '150',
 '1500s',
 '155',
 '15th',
 '16',
 '160',
 '1600',
 '161',
 '16mm',
 '16th',
 '16x9',
 '17',
 '175',
 '1773',
 '17th',
 '18',
 '180',
 '1800s',
 '1839',
 '1869',
 '1871',
 '1888',
 '18th',
 '19',
 '1900',
 '1912',
 '1914',
 '1919',
 '1925',
 '1928',
 '1930',
 '1930s',
 '1932',
 '1933',
 '1935',
 '1937',
 '1938',
 '1939',
 '1940',
 '1940s',
 '1941',
 '1943',
 '1944',
 '1945',
 '1947',
 '1948',
 '1949',
 '1950',
 '1950s',
 '1953',
 '1954',
 '1957',
 '1958',
 '1959',
 '1960',
 '1960s',
 '1961',
 '1962',
 '1963',
 '1964',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1970s',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1980s',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '198

In [23]:
X_count_df = pd.DataFrame(X_count_vec.toarray(),columns = X_names)
X_count_df.shape

(2000, 23784)

In [24]:
X_count_df.head(5)

,00,000,007,05,10,100,1000,100m,101,102,...,zoom,zooming,zooms,zoot,zorg,zorro,zucker,zuko,zwick,zwigoff
0,0,0,0,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#Split data in train ad test sets
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [47]:
X_train,X_test,y_train,y_test = train_test_split(X_count_df,y,test_size = 0.25, random_state = 5)

In [78]:
X_train.shape

(1500, 23784)

In [28]:
 X_test.shape

(500, 23784)

In [29]:
#Use Naive Bayes Classifier
#Using Gaussian
from sklearn.naive_bayes import GaussianNB

In [30]:
gnb = GaussianNB()
model = gnb.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [31]:
metrics.accuracy_score(y_test,y_pred)

0.654

In [32]:
score_model = confusion_matrix(y_test,y_pred)
score_model

array([[173,  85],
       [ 88, 154]], dtype=int64)

In [33]:
accuracy_model = (score_model[0][0]+score_model[1][1])/(sum(score_model[0])+sum(score_model[1]))
accuracy_model

0.654

In [35]:
error_model = (score_model[0][1]+score_model[1][0])/(sum(score_model[0])+sum(score_model[1]))
error_model

0.346

In [36]:
#Using Multinomial because there are multiple features
from sklearn.naive_bayes import MultinomialNB

In [37]:
clf = MultinomialNB()
model2 = clf.fit(X_train,y_train)
y_pred2 = model2.predict(X_test)

In [88]:
model2.predict(test_df)

ValueError: shapes (1,226) and (23784,2) not aligned: 226 (dim 1) != 23784 (dim 0)

In [38]:
metrics.accuracy_score(y_test,y_pred2)

0.798

In [39]:
score_model2 = confusion_matrix(y_test,y_pred2)
score_model2

array([[213,  45],
       [ 56, 186]], dtype=int64)

In [40]:
accuracy_model2 = (score_model2[0][0]+score_model2[1][1])/(sum(score_model2[0])+sum(score_model2[1]))
accuracy_model2

0.798

In [41]:
error_model2 = (score_model2[0][1]+score_model2[1][0])/(sum(score_model2[0])+sum(score_model2[1]))
error_model2

0.202

# For checking the classifcation of a new review, we need to make sure that our model is trained well enough to compensate for the difference in tokens between heterogeneous datasets from different sources. For that, we probably need more data for training. Other than that, we may not be able to correctly classify heterogeneous samples (with different token attributes). 

In [175]:
#Preparing one new sample for clasification after training
def prepare_rev(review):
    review=review.values.flatten()
    review_cat = ''
    for element in review:
        review_cat += element
    
    review_words = review_cat.split(" ")
    
    rev_text = review_words
    rev_one_string = " ".join(rev_text)
    rev_one_string = rev_one_string.replace(' ,',',')
    rev_one_string = rev_one_string.replace(' .',',')
    rev_one_string = rev_one_string.replace("\' ","'")
    rev_one_string = rev_one_string.replace(" \'","'")
    rev_one_string

    rev_list=[]
    rev_list.append(rev_one_string)

    count_vec = CountVectorizer(lowercase = True, stop_words = 'english', min_df = 1)

    X_count_vec = count_vec.fit_transform(rev_list)

    X_names = count_vec.get_feature_names()
    X_names

    X_count_df = pd.DataFrame(X_count_vec.toarray(),columns = X_names)
    X_count_df.shape
    
    common=X_test.columns.intersection(test_df.columns)
    result = pd.DataFrame(0,index=range(1),columns = X_train.columns)
    
    for col in common:
        try:
            result.iloc[0][col]=X_count_df.iloc[0][col]
        except:
            continue
    print(result)
    return result

In [191]:
#Predicting category on new sample based on the trained model
def predict_category(review, model = model2):
    test_df = prepare_rev(review)
    pred = model.predict(test_df)
    if pred == 0:
        return "The review is negative"
    else:
        return "The review is positive"


In [192]:
review = pd.read_table("sample movie review1.txt",header = None)

d:\users\weka\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [193]:
predict_category(review)

   00  000  007  05  10  100  1000  100m  101  102  ...  zoom  zooming  zooms  \
0   0    0    0   0   0    0     0     0    0    0  ...     0        0      0   

   zoot  zorg  zorro  zucker  zuko  zwick  zwigoff  
0     0     0      0       0     0      0        0  

[1 rows x 23784 columns]


'The review is negative'